# Computing the energy of metaphosphate on IBM

## Set experiment parameters

In [ ]:
ibm_computer: str = "ibm_fez"
qubits = (
    list(range(65, 70)) + [77, 78] + list(range(83, 88)) + [96, 97] + list(range(100, 109))
    + [116, 117] + list(range(120, 129)) + [136, 137] + list(range(140, 148))
)
nshots: int = 100_000

len(qubits)

## Setup

In [ ]:
import matplotlib.pyplot as plt; plt.rcParams.update({"font.family": "serif"})
import numpy as np
import pickle

import openfermion as of

import qiskit
from qiskit import qasm2, qasm3
from qiskit_aer import AerSimulator
import qiskit_ibm_runtime
from qiskit_ibm_runtime import SamplerV2 as Sampler

In [ ]:
# Skip this cell if not running on hardware.
service = qiskit_ibm_runtime.QiskitRuntimeService(name="NERSC-US")
computer = service.backend(ibm_computer)
sampler = Sampler(computer)

## Read in the Hamiltonian and ansatz circuit

In [ ]:
data = np.load("hamiltonians/owp_reactant.npz")
print(data)

# Get core energy and one-/two-body tensors
ecore = data['ECORE']
h1 = np.array(data['H1'])
h2 = np.array(data['H2'])
norb = data["NORB"]

# Get number of alpha and beta electrons
n_a = int(data["NELEC"]/2)
n_b = int(data["NELEC"]/2)

h2 = 0.5 * np.asarray(h2.transpose(0, 2, 3, 1), order="C")
h1_new, h2_new = of.chem.molecular_data.spinorb_from_spatial(h1, h2)

# Get the Hamiltonian and transform it to FermionOperator
h = of.InteractionOperator(ecore.item(), h1_new, h2_new)

hamiltonian = of.get_fermion_operator(h)
# hamiltonian.compress(0.001)

nqubits = of.utils.count_qubits(hamiltonian)
nterms = len(hamiltonian.terms)

print(f"Hamiltonian acts on {nqubits} qubit(s) and has {nterms} term(s).")

In [ ]:
# hamiltonian = of.jordan_wigner(hamiltonian)

In [ ]:
nqubits = of.utils.count_qubits(hamiltonian)
nterms = len(hamiltonian.terms)

print(f"Hamiltonian acts on {nqubits} qubit(s) and has {nterms} term(s).")

In [ ]:
# hamiltonian_cirq = of.qubit_operator_to_pauli_sum(hamiltonian)

In [ ]:
circuit_dir = "circuits/"
circuit_fname = "owp_circuit.qasm"
# circuit_fname = "owp_circuit_2.qasm"
# circuit_fname = "owp_circuit_3.qasm"  # _compiled_bqskit.qasm"
# circuit_fname = "po3_1w-22o_reactant.qasm"
# circuit_fname = "po3_2w-28o_reactant.qasm"
# circuit_fname = "po3_2w-28o_product.qasm"

In [ ]:
circuit = qasm3.load(circuit_dir + circuit_fname)
# circuit.draw(fold=-1)

In [ ]:
dag = circuit_to_dag(circuit)
dag.idle_wires()
dag.remove_qubits(*list(dag.idle_wires()))
circuit = dag_to_circuit(dag)

In [ ]:
base = circuit.copy()
base = qiskit.transpiler.passes.RemoveBarriers()(base)

In [ ]:
base.count_ops()

## Prepare to run on hardware

In [ ]:
to_run = base.copy()
to_run.measure_active()
to_run = [to_run]

In [ ]:
to_run = base.copy()
to_run = [to_run]

for c in to_run:
    c.measure_all()

In [ ]:
to_run = qiskit.transpile(
    to_run,
    optimization_level=3,
    backend=computer,
    initial_layout=qubits,
    seed_transpiler=123
)

In [ ]:
for c in to_run:
    print(c.count_ops())

In [ ]:
to_run[0].depth()

In [ ]:
ops = list(to_run[0].count_ops().keys())
num = list(to_run[0].count_ops().values())

In [ ]:
sum([number for op, number in to_run[0].count_ops().items() if op != "cz"])

In [ ]:
plt.bar(ops, num, color="green", edgecolor="black")
plt.xlabel("Gate")
plt.ylabel("Count");

In [ ]:
# to_run[0].draw(fold=-1, idle_wires=False)

## Run on exact simulator

In [ ]:
simulator = AerSimulator(method="matrix_product_state")
result = simulator.run(to_run, shots=10)
counts = result.result().get_counts()
counts

In [ ]:
hartree_fock_bitstring = list(counts.keys())[0]
hartree_fock_bitstring

## Run on noisy simulator

In [ ]:
sim = AerSimulator.from_backend(computer, method="matrix_product_state")

In [ ]:
result = sim.run(to_run, shots=nshots)
counts = result.result().get_counts()

In [ ]:
qiskit.visualization.plot_histogram(
    counts,
    target_string=hartree_fock_bitstring,
    sort="hamming",
    number_to_keep=10,
    figsize=(7, 8),
    title=sim.name,
)

In [ ]:
# pickle.dump(counts, open(f"owp_counts_{sim.name}.pkl", "wb"))

In [ ]:
# counts = pickle.load(open(f"owp_counts_{sim.name}.pkl", "rb"))

## Run on hardware

In [ ]:
job = sampler.run(to_run, shots=nshots)
# job = service.job("d4vi5hfg0u6s73daqh3g")

In [ ]:
all_counts_hardware = []

In [ ]:
res = job.result()

In [ ]:
for r in res:
    all_counts_hardware.append(r.data.meas.get_counts())

In [ ]:
len(all_counts_hardware[0])

In [ ]:
hartree_fock_bitstring in all_counts_hardware[0].keys()

In [ ]:
import qiskit.visualization


qiskit.visualization.plot_histogram(
    all_counts_hardware[0],
    target_string=hartree_fock_bitstring,
    sort="hamming",
    number_to_keep=10,
    figsize=(7, 8),
    title=computer.name,
)

## Save data

In [ ]:
import datetime


time_key = datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
fname = circuit_fname + f"_counts_{computer.name}_nshots_{nshots}_{time_key}"
fname

In [ ]:
pickle.dump(all_counts_hardware[0], open(fname, "wb"))

## Run SQD

In [ ]:
import numpy as np
import os

from sqd_functions import run_sqd


file = os.path.expanduser(f'./hamiltonians/owp_reactant.npz')

integrals_data = np.load(file, allow_pickle=True)
h1 = integrals_data['H1']
h2 = integrals_data['H2']
n_orbitals = integrals_data['NORB']
num_electrons = integrals_data['NELEC']
ecore = integrals_data['ECORE']

f1 = "./results/fez/owp_circuit.qasm_counts_ibm_fez_nshots_100000_2025_12_28_16:10:39"
f2 = "./results/fez/owp_circuit_2.qasm_counts_ibm_fez_nshots_100000_2025_12_28_18:44:58"

for experiment_iter in range(2):
    for samples_per_batch in [100, 1000]:
        print("Samples per batch:", samples_per_batch)

        result = run_sqd(
            one_body_integrals=h1,
            two_body_integrals=h2,
            n_orbitals=n_orbitals,
            num_electrons=(num_electrons // 2, num_electrons // 2),
            spin_sq=0.0,
            nuclear_repulsion_energy=0.0,
            e_core=ecore,
            num_batches=5,
            samples_per_batch=samples_per_batch,
            max_config_recovery_iterations=5,
            max_davidson_cycles=10**4,
            symmetrize_spin=True,
            sampler=None,
            force_fci=False, # set this to True to skip circuit sampling and just use all bitstrings in FCI space.
            bitstrings_files=[f1, f2],
            save_dir=None,
        )
        print(f'Result for {samples_per_batch} samples per batch:')
        print(result.energy)
        print(f'total energy: {result.energy + ecore} Hartree')


-635.6730264251646 Hartree